In [807]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np

In [808]:
train = pd.read_excel("/Users/aaron/Downloads/타이타닉_data.xlsx", sheetname="train")
test = pd.read_excel("/Users/aaron/Downloads/타이타닉_data.xlsx", sheetname="test")

In [809]:
train["train"]=1
test["train"]=0

In [810]:
data = pd.concat([train,test],0)

In [811]:
data['Title'] = data.name.str.extract(' ([A-Za-z]+)\.')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [812]:
#Mlle -> 프랑스어로 Miss
#Mme -> 마담, 부인
#Don -> 스페인어로 남성
#Dona -> 스페인어로 여성
#Dr -> doctor
#Rev -> 목사에 대한 극존칭
#Major -> 소령
#Jonkheer -> 귀족
#Capt -> 함장
#Col -> 대령
#Countess -> 귀족
data["Title"] = data["Title"].replace(['Countess', 'Dr', 'Jonkheer', 'Rev',"Lady","Master"], 'Other')
data['Title'] = data['Title'].replace('Don', 'Mr')
data['Title'] = data['Title'].replace('Dona', 'Mrs')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
#data['Title'] = data['Title'].replace('Lady', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data["Title"] = data["Title"].replace(["Major","Col","Capt" ,'Sir'], 'Navy')

In [813]:
data["family"] = data["sibsp"]+data["parch"]
data.loc[data['family'] > 0, 'travelled_alone'] = 'No'
data.loc[data['family'] == 0, 'travelled_alone'] = 'Yes'

In [814]:
data['pclass'] = data['pclass'].astype(str)

In [815]:
ages = pd.cut(data["age"], [0,18,35,46,60,100], labels=False, retbins=True, right=False)

In [816]:
data["emb_age"] = ages[0]
data['emb_age'] = data['emb_age'].astype(str)

In [1027]:
data["emb"] = data["pclass"] + data["sex"] + data["travelled_alone"] + data['emb_age']

In [1028]:
data

,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,survival,train,Title,family,travelled_alone,emb_age,emb,seq
0,2,"Lehmann, Miss. Bertha",Female,0.2125,0,0,SC 1748,0.023422,C,1,1,Miss,0,Yes,0,2FemaleYes0,52
1,3,"Laitinen, Miss. Kristina Sofia",Female,0.4625,0,0,4135,0.018718,S,0,1,Miss,0,Yes,2,3FemaleYes2,45
2,3,"Rosblom, Mr. Viktor Richard",Male,0.2250,1,1,370129,0.039447,S,0,1,Mr,2,No,1,3MaleNo1,7
3,3,"Karaic, Mr. Milan",Male,0.3750,0,0,349246,0.015420,S,0,1,Mr,0,Yes,1,3MaleYes1,1
4,3,"Moen, Mr. Sigurd Hansen",Male,0.3125,0,0,348123,0.014932,S,0,1,Mr,0,Yes,1,3MaleYes1,1
5,3,"Danoff, Mr. Yoto",Male,0.3375,0,0,349219,0.015420,S,0,1,Mr,0,Yes,1,3MaleYes1,1
6,3,"Peacock, Master. Alfred Edward",Male,0.0125,1,1,SOTON/O.Q. 3101315,0.026897,S,0,1,Other,2,No,0,3MaleNo0,4
7,1,"Payne, Mr. Vivian Ponsonby",Male,0.2875,0,0,12749,0.182500,S,0,1,Mr,0,Yes,1,1MaleYes1,12
8,3,"Dennis, Mr. Samuel",Male,0.2750,0,0,A/5 21172,0.014151,S,0,1,Mr,0,Yes,1,3MaleYes1,1
9,2,"Brown, Miss. Amelia ""Mildred""",Female,0.3000,0,0,248733,0.025374,S,1,1,Miss,0,Yes,1,2FemaleYes1,21


In [1029]:
data["emb"].nunique()

56

In [1030]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def normalize_col(data, colum):
    scaler = MinMaxScaler()
    data[colum] = scaler.fit_transform(data[colum].values.reshape(-1,1))
    return data
data= normalize_col(data,"fare")
data= normalize_col(data,"age")
#data= normalize_col(data,"family")

In [1031]:
tok = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level="")

In [1032]:
tok.fit_on_texts(data["emb"].values)

In [1033]:
seq = tok.texts_to_sequences(data["emb"].values)

In [1034]:
seq = np.array(seq)

In [1035]:
data["seq"] = seq

In [1036]:
data["seq"].nunique()

56

In [1037]:
train_x = data[data["train"]==1][["seq", "fare", "age"]]
train_y = data[data["train"]==1]["survival"]

In [1038]:
test_x = data[data["train"]==0][["seq", "fare", "age"]]
test_y = data[data["train"]==0]["survival"]

In [1039]:
train_x

,seq,fare,age
0,47,0.023422,0.2125
1,40,0.018718,0.4625
2,9,0.039447,0.2250
3,1,0.015420,0.3750
4,1,0.014932,0.3125
5,1,0.015420,0.3375
6,3,0.026897,0.0125
7,14,0.182500,0.2875
8,1,0.014151,0.2750
9,13,0.025374,0.3000


In [1080]:
input1 = tf.keras.layers.Input(shape=(2,), name="num")
input2 = tf.keras.layers.Input(shape=(1), name="emb")

In [1081]:
emb_layer = tf.keras.layers.Embedding(57,20, embeddings_regularizer="l2")(input2)

In [1082]:
#drop1 = tf.keras.layers.SpatialDropout1D(0.3)(emb_layer)

In [1083]:
emb_layer = tf.keras.layers.Flatten()(emb_layer)

In [1084]:
concat_layer = tf.keras.layers.concatenate([input1,emb_layer])

In [1085]:
dense1 = tf.keras.layers.Dense(50,kernel_initializer="glorot_normal",use_bias=True)(concat_layer)

bat = tf.keras.layers.BatchNormalization()(dense1)

#dense2 = tf.keras.layers.Dense(50, 
#                                kernel_initializer="glorot_normal", kernel_regularizer="l2", 
#                             use_bias=True)(dense1)
re = tf.keras.layers.ReLU()(bat)
dense1 = tf.keras.layers.Dense(50,kernel_initializer="glorot_normal",use_bias=True)(re)

bat = tf.keras.layers.BatchNormalization()(dense1)

#dense2 = tf.keras.layers.Dense(50, 
#                                kernel_initializer="glorot_normal", kernel_regularizer="l2", 
#                             use_bias=True)(dense1)
re = tf.keras.layers.ReLU()(bat)

dense3 = tf.keras.layers.Dense(10, 
                                kernel_initializer="glorot_normal",
                             use_bias=True)(re)


out = tf.keras.layers.Dense(1, activation="sigmoid",
                             kernel_initializer="glorot_normal",use_bias=True)(dense3)

In [1086]:
model = tf.keras.Model(inputs = [input1, input2], outputs = out)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [1088]:
model.fit([train_x[["fare","age"]].values, train_x["seq"].values],train_y.values, batch_size=32,
          epochs=100, validation_data=([test_x[["fare","age"]].values, test_x["seq"].values],test_y.values))

Train on 730 samples, validate on 313 samples
Epoch 1/100
730/730 [==============================] - 0s 126us/sample - loss: 0.3604 - accuracy: 0.8370 - val_loss: 0.5566 - val_accuracy: 0.7827
Epoch 2/100
730/730 [==============================] - 0s 119us/sample - loss: 0.3610 - accuracy: 0.8411 - val_loss: 0.5841 - val_accuracy: 0.7955
Epoch 3/100
730/730 [==============================] - 0s 122us/sample - loss: 0.3657 - accuracy: 0.8315 - val_loss: 0.5782 - val_accuracy: 0.8051
Epoch 4/100
730/730 [==============================] - 0s 121us/sample - loss: 0.3676 - accuracy: 0.8329 - val_loss: 0.5747 - val_accuracy: 0.7955
Epoch 5/100
730/730 [==============================] - 0s 125us/sample - loss: 0.3596 - accuracy: 0.8342 - val_loss: 0.5755 - val_accuracy: 0.7764
Epoch 6/100
730/730 [==============================] - 0s 119us/sample - loss: 0.3636 - accuracy: 0.8397 - val_loss: 0.5855 - val_accuracy: 0.8115
Epoch 7/100
730/730 [==============================] - 0s 122us/sample -

730/730 [==============================] - 0s 125us/sample - loss: 0.3562 - accuracy: 0.8411 - val_loss: 0.6100 - val_accuracy: 0.7891
Epoch 57/100
730/730 [==============================] - 0s 120us/sample - loss: 0.3428 - accuracy: 0.8466 - val_loss: 0.6280 - val_accuracy: 0.7955
Epoch 58/100
730/730 [==============================] - 0s 121us/sample - loss: 0.3539 - accuracy: 0.8356 - val_loss: 0.6089 - val_accuracy: 0.7859
Epoch 59/100
730/730 [==============================] - 0s 115us/sample - loss: 0.3441 - accuracy: 0.8384 - val_loss: 0.5965 - val_accuracy: 0.7923
Epoch 60/100
730/730 [==============================] - 0s 114us/sample - loss: 0.3431 - accuracy: 0.8438 - val_loss: 0.6418 - val_accuracy: 0.7796
Epoch 61/100
730/730 [==============================] - 0s 118us/sample - loss: 0.3559 - accuracy: 0.8356 - val_loss: 0.6387 - val_accuracy: 0.7891
Epoch 62/100
730/730 [==============================] - 0s 122us/sample - loss: 0.3415 - accuracy: 0.8329 - val_loss: 0.6641 